In [1]:
%run utils.ipynb

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error

In [3]:
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [4]:
df=pd.read_csv("../data/train.csv")

In [5]:
#drop some useless columns
cols_to_drop=["Id","PID"]
df=df.drop(columns=cols_to_drop)

Let's look at the perecentage of null in each feature.

In [6]:
df_null=df.isnull().mean().sort_values(ascending=False)*100
df_null[df_null>0]

Pool QC           99.561190
Misc Feature      96.830814
Alley             93.174061
Fence             80.497318
Fireplace Qu      48.756704
Lot Frontage      16.089712
Garage Finish      5.558264
Garage Qual        5.558264
Garage Yr Blt      5.558264
Garage Cond        5.558264
Garage Type        5.509508
Bsmt Exposure      2.827889
BsmtFin Type 2     2.730375
Bsmt Cond          2.681619
Bsmt Qual          2.681619
BsmtFin Type 1     2.681619
Mas Vnr Area       1.072647
Mas Vnr Type       1.072647
Bsmt Full Bath     0.097513
Bsmt Half Bath     0.097513
Garage Area        0.048757
Garage Cars        0.048757
Total Bsmt SF      0.048757
Bsmt Unf SF        0.048757
BsmtFin SF 2       0.048757
BsmtFin SF 1       0.048757
dtype: float64

#### Imputesome missing values

In [7]:
cols_to_impute_with_none=["Pool QC","Misc Feature","Alley","Fence","Fireplace Qu",
                          "Garage Finish","Garage Qual","Garage Cond","Garage Type",
                         "Bsmt Exposure","BsmtFin Type 2","Bsmt Cond","Bsmt Qual","BsmtFin Type 1",
                         "Mas Vnr Type"]

cols_to_impute_with_zero=["Garage Yr Blt","Mas Vnr Area","Bsmt Full Bath","Bsmt Half Bath","Garage Area",
                         "Garage Cars","Total Bsmt SF","Bsmt Unf SF","BsmtFin SF 2","BsmtFin SF 1"]

In [8]:
for i in cols_to_impute_with_none:
    df[i]=df[i].fillna("None")
    
for j in cols_to_impute_with_zero:
    df[j]=df[j].fillna(0)

In [9]:
#Replace Lot Frontage with the median of the Neighborhood
df['Lot Frontage']=df.groupby(by="Neighborhood")['Lot Frontage'].transform(lambda x: x.fillna(x.median()))

In [10]:
#Should show as 0, to ensure no more nulls
df.isnull().sum().sum()

3

Uhh, there are still 3?

In [11]:
df[df['Lot Frontage'].isnull()]

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
463,120,RM,NaN,9763,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,GrnHill,Norm,Norm,TwnhsE,1Story,7,5,1998,1998,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,239.0,Gd,TA,PConc,Gd,TA,No,GLQ,1102.0,ALQ,72.0,328.0,1502.0,GasA,Ex,Y,SBrkr,1502,0,0,1502,1.0,0.0,1,1,1,1,Gd,5,Typ,1,TA,Attchd,1998.0,Fin,1.0,286.0,TA,TA,Y,0,0,0,0,0,0,None,None,None,0,9,2007,WD,330000
1121,160,RH,NaN,3612,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Landmrk,Norm,Norm,Twnhs,2Story,6,5,1993,1994,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,338.0,Unf,0.0,292.0,630.0,GasA,Ex,Y,SBrkr,630,690,0,1320,0.0,0.0,2,1,3,1,TA,5,Typ,0,None,Detchd,1993.0,Unf,2.0,484.0,TA,TA,Y,0,48,0,0,0,0,None,MnPrv,None,0,6,2006,WD,137000
1211,120,RM,NaN,8239,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,GrnHill,Norm,Norm,TwnhsE,1Story,7,5,1986,1986,Gable,CompShg,BrkFace,Wd Sdng,None,0.0,Gd,TA,CBlock,None,None,None,None,0.0,None,0.0,0.0,0.0,GasA,Gd,Y,SBrkr,1295,0,0,1295,0.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1986.0,RFn,1.0,312.0,TA,TA,Y,0,0,0,0,0,0,None,None,None,0,11,2006,WD,230000


In [12]:
df['Neighborhood'].value_counts().tail(2)

GrnHill    2
Landmrk    1
Name: Neighborhood, dtype: int64

As we see here, GrnHill and Landmark neighborhoods are rare data points and no Lot Frontage info is available.  
We'll just impute Lot Frontage with median of the whole dataset.

In [13]:
df['Lot Frontage']=df['Lot Frontage'].fillna(df['Lot Frontage'].median())

In [14]:
#Should show as 0, to ensure no more nulls
df.isnull().sum().sum()

0

Ok! everything looks good now.

### We will change ordinal features to numbers which suggests a ranking.

In [15]:
var='Lot Shape'
df[var]=df[var].map({"IR3":1,"IR2":2,"IR1":3,"Reg":4})

var='Utilities'
df[var]=df[var].map({"ELO":1,"NoSeWa":2,"NoSewr":3,"AllPub":4})

var='Land Slope'
df[var]=df[var].map({"Sev":1,"Mod":2,"Gtl":3})

var='Exter Qual'
df[var]=df[var].map({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

var='Exter Cond'
df[var]=df[var].map({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

var='Bsmt Qual'
df[var]=df[var].map({"None":0,"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

var='Bsmt Cond'
df[var]=df[var].map({"None":0,"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

var='Bsmt Exposure'
df[var]=df[var].map({"None":0,"No":1,"Mn":2,"Av":3,"Gd":4})

var='BsmtFin Type 1'
df[var]=df[var].map({"None":0,"Unf":1,"LwQ":2,"Rec":3,"BLQ":4,"ALQ":5,"GLQ":6})

var='BsmtFin Type 2'
df[var]=df[var].map({"None":0,"Unf":1,"LwQ":2,"Rec":3,"BLQ":4,"ALQ":5,"GLQ":6})

var='Heating QC'
df[var]=df[var].map({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

var='Electrical'
df[var]=df[var].map({"Mix":1,"FuseP":2,"FuseF":3,"FuseA":4,"SBrkr":5})

var='Kitchen Qual'
df[var]=df[var].map({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

var='Functional'
df[var]=df[var].map({"Sal":1,"Sev":2,"Maj2":3,"Maj1":4,"Mod":5,"Min2":6,
                    "Min1":7,"Typ":8})

var='Fireplace Qu'
df[var]=df[var].map({"None":0,"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

var='Garage Finish'
df[var]=df[var].map({"None":0,"Unf":1,"RFn":2,"Fin":3})

var='Garage Qual'
df[var]=df[var].map({"None":0,"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

var='Garage Cond'
df[var]=df[var].map({"None":0,"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5})

var='Paved Drive'
df[var]=df[var].map({"N":1,"P":2,"Y":3})

var='Pool QC'
df[var]=df[var].map({"None":0,"Fa":1,"TA":2,"Gd":3,"Ex":4})

var='Fence'
df[var]=df[var].map({"None":0,"MnWw":1,"GdWo":2,"MnPrv":3,"GdPrv":4})

var='Central Air'
df[var]=df[var].map({"N":0,"Y":1})

In [16]:
#get the categorical features
df_c=df.select_dtypes(exclude='number')

In [17]:
df_c=pd.get_dummies(df_c)

In [19]:
df_c.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 158 columns):
 #    Column                Dtype
---   ------                -----
 0    MS Zoning_A (agr)     uint8
 1    MS Zoning_C (all)     uint8
 2    MS Zoning_FV          uint8
 3    MS Zoning_I (all)     uint8
 4    MS Zoning_RH          uint8
 5    MS Zoning_RL          uint8
 6    MS Zoning_RM          uint8
 7    Street_Grvl           uint8
 8    Street_Pave           uint8
 9    Alley_Grvl            uint8
 10   Alley_None            uint8
 11   Alley_Pave            uint8
 12   Land Contour_Bnk      uint8
 13   Land Contour_HLS      uint8
 14   Land Contour_Low      uint8
 15   Land Contour_Lvl      uint8
 16   Lot Config_Corner     uint8
 17   Lot Config_CulDSac    uint8
 18   Lot Config_FR2        uint8
 19   Lot Config_FR3        uint8
 20   Lot Config_Inside     uint8
 21   Neighborhood_Blmngtn  uint8
 22   Neighborhood_Blueste  uint8
 23   Neighborhood_BrDale   uint8
 24   Ne